# Линейные модели в классификации текстов
Основано на материалах подготовленных Ильей Ирхиным

In [ ]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
import heapq

import numpy as np

from IPython.display import Image
from sklearn.datasets import fetch_20newsgroups

## Посмотрим на данные

In [ ]:
dataset = fetch_20newsgroups()

### Пример элемента выборки

In [ ]:
print('Features\n\n', dataset['data'][0])
print('Target\n\n', dataset.target[0])

### Категории

In [ ]:
dataset.target_names

### Train и test выборки

Возьмём 2 темы из одного раздела, возможно, их будет сложно отличить друг от друга

In [ ]:
categories = [
    'rec.sport.baseball',
    'rec.sport.hockey'
]
train_data = fetch_20newsgroups(subset='train',
                                categories=categories,
                                remove=('headers', 'footers', 'quotes'))
test_data = fetch_20newsgroups(subset='test',
                               categories=categories,
                               remove=('headers', 'footers', 'quotes'))

In [ ]:
print('# examples in train', len(train_data.data))
print('# examples in test', len(test_data.data))

## Приведем данные к виду, с которым можно работать

### Векторизация

Для векторизации текстов воспользуемся CountVectorizer, он представляет документ как мешок слов. Можно всячески варировать извлечение признаков (убирать редкие слова, убирать частые слова, убирать слова общей лексики, брать биграмы и т.д.)

In [ ]:
Image('pics/bag_of_words.png', width=600)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
CountVectorizer()

In [ ]:
count_vectorizer = CountVectorizer(min_df=3, ngram_range=(1, 2)) 

Мы будем работать с эффективным представлением матриц "Compressed Sparse Row Format" (CSR). Почитать о формате можно здесь: https://en.wikipedia.org/wiki/Sparse_matrix#Compressed_sparse_row_(CSR,_CRS_or_Yale_format)

In [ ]:
sparse_feature_matrix = count_vectorizer.fit_transform(train_data.data)
dense_feature_matrix = sparse_feature_matrix.toarray()

In [ ]:
print('Dense matrix shape', dense_feature_matrix.data.shape)
print('Sparse matrix shape', sparse_feature_matrix.shape)

In [ ]:
%%time
dense_feature_matrix.sum()

In [ ]:
%%time
sparse_feature_matrix.sum()

### Словарь токен-слово

Соберем обратный словарь, в котором каждому токену (номеру) будет сопоставлено слово из оригинального словаря

In [ ]:
num_2_words = {
    v: k
    for k, v in count_vectorizer.vocabulary_.items()
}

## Начнем обучать линейные модели

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

### Обучим логистическую регрессию предсказывать тему документа

In [ ]:
algo = LogisticRegression()
algo.fit(dense_feature_matrix, train_data.target)

Слова с наименьшим отрицательным весом являются характерными словами класса 0

Слова с наибольшим положительным весом являются характерными словами класса 1

In [ ]:
W = algo.coef_.shape[1]
NUM_WORDS = 10
class_2_function = {'Baseball': heapq.nsmallest, 'Hockey': heapq.nlargest}

for category, function in class_2_function.items():
    topic_words = [
        num_2_words[w_num]
        for w_num in function(NUM_WORDS, range(W), key=lambda w: algo.coef_[0, w])
    ]
    print(category)
    print(', '.join(topic_words))

### Оценим качество

Сравним качество на фолдах с качеством на трейне и на отложенном тесте

In [ ]:
algo = LogisticRegression()
arr = cross_val_score(algo, dense_feature_matrix, train_data.target, cv=5, scoring='accuracy')
print(arr)
print(np.mean(arr))

Почему это неправильная кроссвалидация?

In [ ]:
algo.fit(dense_feature_matrix, train_data.target)

In [ ]:
print('Train accuracy', accuracy_score(algo.predict(dense_feature_matrix), train_data.target))
print('Test accuracy', accuracy_score(algo.predict(count_vectorizer.transform(test_data.data)), test_data.target))

Мы видим переобучение, почему?

### Регуляризация

Добавим l1 - регуляризатор с коэффициентом 0.1

In [ ]:
algo = LogisticRegression(penalty='l1', C=0.1)
arr = cross_val_score(algo, dense_feature_matrix, train_data.target, cv=5, scoring='accuracy')
print(arr)
print(np.mean(arr))

In [ ]:
algo.fit(sparse_feature_matrix, train_data.target)

In [ ]:
print('Train accuracy', accuracy_score(algo.predict(dense_feature_matrix), train_data.target))
print('Test accuracy', accuracy_score(algo.predict(count_vectorizer.transform(test_data.data)), test_data.target))

Добавление регуляризатора уменьшает отличие на трейне и тесте, но ухудшает качество.

## Pipeline

Чтобы не делать векторизацию и обучение раздельно, есть класс Pipeline. Он позволяет объединить в цепочку последовательность действий.

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
pipeline = Pipeline([("vectorizer", CountVectorizer(min_df=3, ngram_range=(1, 2))),
                     ("algo", LogisticRegression())])

In [ ]:
pipeline.fit(train_data.data, train_data.target)

In [ ]:
print('Train accuracy', accuracy_score(pipeline.predict(train_data.data), train_data.target))
print('Test accuracy', accuracy_score(pipeline.predict(test_data.data), test_data.target))

Значения примерно такие же как мы получали ранее, делая шаги раздельно.

In [ ]:
from sklearn.pipeline import make_pipeline

При кроссвалидации нужно, чтобы CountVectorizer не обучался на тесте. Pipeline позволяет это сделать просто.

In [ ]:
pipeline = make_pipeline(CountVectorizer(min_df=3, ngram_range=(1, 2)),
                         LogisticRegression())
arr = cross_val_score(pipeline, train_data.data, train_data.target, cv=5, scoring='accuracy')
print(arr)
print(np.mean(arr))

In [ ]:
pipeline = make_pipeline(CountVectorizer(min_df=3, ngram_range=(1, 2)),
                         LogisticRegression())
arr = cross_val_score(pipeline, train_data.data, train_data.target, cv=3, scoring='accuracy')
print(arr)
print(np.mean(arr))

В Pipeline можно добавлять новые шаги препроцессинга данных.

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
Image('pics/tfidf.png')

Подробнее про tf-idf можно прочитать здесь: https://ru.wikipedia.org/wiki/TF-IDF

In [ ]:
pipeline = make_pipeline(CountVectorizer(min_df=3, ngram_range=(1, 2)),
                         TfidfTransformer(),
                         LogisticRegression())
arr = cross_val_score(pipeline, train_data.data, train_data.target, cv=5, scoring='accuracy')
print(arr)
print(np.mean(arr))

In [ ]:
pipeline.fit(train_data.data, train_data.target)

In [ ]:
print('Train accuracy', accuracy_score(pipeline.predict(train_data.data), train_data.target))
print('Test accuracy', accuracy_score(pipeline.predict(test_data.data), test_data.target))